# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
cause_list = ["falta de pastos?", "falta ?de agua", "estr[eé]ss? térmico",
              "enfermedades", "robos", "desapariciones", "predadores",
              "otros", "sección iv"]
translation_list = ["lack_of_pasture", "lack_of_water", "heat_stress",
                    "diseases", "theft", "disappearancse", "predators",
                    "other"]
option_list = ["yes, always", "no, never", "sometimes"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
grab_list = []

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_section_3_3(section_search, cause_list,
                      translation_list, option_list):

    section_3_3_dict = {}

    if len(section_search) == 1:
        section_string = section_search[0] + "sección iv"

        for i in range(len(cause_list)-1):
            item_1 = cause_list[i]
            item_2 = cause_list[i+1]
            term = translation_list[i]

            row_search = find_between(item_1, item_2, section_string)

            if len(row_search) == 1:
                row_string = row_search[0]

                selection_regex = r"[|i][_ ]*([x*]*)[_ ]*[|i]"
                selection_search = re.findall(selection_regex, row_string)

                if len(selection_search) == 3:
                    match_regex = r"[x*]"

                    if re.match(match_regex, selection_search[0]):
                        section_3_3_dict[term] = option_list[0]
                    elif re.match(match_regex, selection_search[1]):
                        section_3_3_dict[term] = option_list[1]
                    elif re.match(match_regex, selection_search[2]):
                        section_3_3_dict[term] = option_list[2]
                    else:
                        check_regex = "sí[^\n]*\n"
                        check_search = re.findall(check_regex, row_string)
                        check_string = check_search[0]

                        drop_list = [
                            "sí, siempre |__|\t\t      no, nunca |__|\t\ta veces |__|\n",
                            "sí, siempre |__|\t      no, nunca |__|\ta veces |__|\n",
                            "sí, siempre |__|\t  no, nunca |__|\t       a veces |__|\n",
                            "sí, siempre |_  |\t\t      no, nunca |__|\t\ta veces |  _|\n",
                            "sí, siempre |__|\t\tno, nunca |__|\t\ta veces |__|\n",
                            "sí, siempre |_|\t      no, nunca |__|\ta veces |_| \n",
                            "sí, siempre | |\t      no, nunca |__|\ta veces |_| \n",
                            "sí, siempre |_ _|\t      no, nunca |__|\ta veces |__|\n",
                            "sí, siempre |__|\t      no, nunca |__|\ta veces |_ |\n",
                            "sí, siempre |__|\t\xa0 \xa0 \xa0 no, nunca |__|\ta veces |__|\n",
                            "sí, siempre |__|\t\t      no, nunca |__|\t\ta veces |  _|\n",
                            "sí, siempre |__|\t\t      no, nunca |__|\t\ta veces |_ |\n"
                        ]

                        fix_list = []

                        if check_string in drop_list:
                            section_3_3_dict[term] = "NONE SELECTED"
                        elif check_string in fix_list:
                            section_3_3_dict[term] = "CHECK THIS"
                        else:
                            section_3_3_dict[term] = "UNENCOUNTERED FORMAT 3"
                else:
                    section_3_3_dict[term] = "UNENCOUNTERED FORMAT 2"
            else:
                section_3_3_dict[term] = "UNENCOUNTERED FORMAT"
    else:
        section_3_3_dict = [{}]

    # capture other field
    if len(section_search) == 1:
        section_string = section_search[0] + "sección iv"

        description_start = "especificar\)"
        description_end = "sí"
        other_search = find_between(description_start, description_end,
                                    section_string)

        if len(other_search) == 1:
            other_string = other_search[0]
            other_string = other_string.strip()

            section_3_3_dict["other_description"] = other_string
        else:
            section_3_3_dict["other_description"] = "UNENCOUNTERED FORMAT"

    return section_3_3_dict

### Process the Whole File

In [ ]:
def process_files(source_directory, cause_list,
                  translation_list, option_list):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            data_dict = {}

            section_start = "iii-3"
            section_end = "sección iv"
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            section_3_3_dict = parse_section_3_3(section_search, cause_list,
                                                 translation_list, option_list)

            # check to make sure files contain the right amount of entries
            if len(section_3_3_dict) != 9:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_dict: {len(section_3_3_dict)}\n")

            data_dict = {}
            data_dict["filename"] = process_file
            data_dict["interviewee"] = interviewee

            for key, value in section_3_3_dict.items():
                data_dict[key] = value

            dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("NONE SELECTED", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 9
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, cause_list,
                       translation_list, option_list)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

## Load
### Export to CSV

In [ ]:
datasets_directory = "../datasets/"
filename = "section_3_3.csv"
file_path = os.path.join(datasets_directory, filename)

clean_df.to_csv(file_path, index=False, na_rep="null")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(10)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]